In [1]:
import pandas as pd
import json

In [2]:
FILE = "../data/AMAZON_FASHION_5.json"

In [3]:
df = pd.DataFrame()

In [ ]:
with open(FILE, 'r') as f:
    for json_line in f.readlines():
        line = json.loads(json_line)
        if line.get("style"):
            for key, value in line.pop("style").items():
                line[f"style.{key}"] = value
        df = df.append(line, ignore_index=True)

In [5]:
reviews_by_user = df.groupby(by="reviewerID").count()

KeyError: 'reviewerID'

In [64]:
reviews_by_user.max()

overall             16
verified            16
reviewTime          16
asin                16
reviewerName        16
reviewText          16
summary             16
unixReviewTime      16
style.Size:         16
style.Color:        16
style.Size Name:     1
style.Style:         1
vote                16
image               16
dtype: int64

In [65]:
reviews_by_user.min()

overall             4
verified            4
reviewTime          4
asin                4
reviewerName        4
reviewText          0
summary             4
unixReviewTime      4
style.Size:         0
style.Color:        0
style.Size Name:    0
style.Style:        0
vote                0
image               0
dtype: int64

In [66]:
df.to_json("../data/AMAZON_FASION_5_formatted.json")

In [6]:
df = pd.read_json("../data/AMAZON_FASION_5_formatted.json")

In [10]:

from IPython.core.display import HTML

display(HTML(df[df.duplicated(subset=["reviewerID", "asin"])].to_html()))

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style.Size:,style.Color:,style.Size Name:,style.Style:,vote,image
1,5,1,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,Tonya B.,Great product and price!,Five Stars,1441324800,Big Boys,Black (37467610) / Red/White,None,None,NaN,None
2,5,1,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,Tonya B.,Great product and price!,Five Stars,1441324800,Big Boys,Blue/Gray Logo,None,None,NaN,None
3,5,1,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,Tonya B.,Great product and price!,Five Stars,1441324800,Big Boys,Blue (37867638-99) / Yellow,None,None,NaN,None
4,5,1,"09 4, 2015",ALJ66O1Y6SLHA,B000K2PJ4K,Tonya B.,Great product and price!,Five Stars,1441324800,Big Boys,Blue/Pink,None,None,NaN,None
6,5,1,"05 6, 2015",A3W11493KS6Z2L,B000K2PJ4K,NaeNae,Stays vibrant after many washes,Great,1430870400,Little Boys,Blue/Orange,None,None,NaN,None
7,5,1,"05 6, 2015",A3W11493KS6Z2L,B000K2PJ4K,NaeNae,Stays vibrant after many washes,Good,1430870400,Little Boys,Blue (37867638-99) / Yellow,None,None,NaN,None
8,5,1,"05 6, 2015",A3W11493KS6Z2L,B000K2PJ4K,NaeNae,My son really likes the pink. Ones which I was nervous about,Great,1430870400,Little Boys,Blue/Pink,None,None,NaN,None
9,3,1,"05 6, 2015",A3W11493KS6Z2L,B000K2PJ4K,NaeNae,Waaay too small. Will use for future child.,Oops!,1430870400,Little Boys,Light Blue/Red,None,None,NaN,None
13,5,1,"08 22, 2016",AW8UBYMNJ894V,B000KPIHQ4,Cognizant Consumer,"Pinnacle seems to have more cushioning so my husband likes them better. He's tried all the other Powerstep inserts as well as other brands. This is his must have insert. He is 6'3"" and weighs 257lbs with bone spurs (plantar fasciitis) and a high instep. He is very active and wears these in all his shoes. His bone spurs rarely bother him any more. Remember to replace them every 3 months if you wear them every day, especially if you weigh more than average and/or run.",Personal favorite,1471824000,None,None,None,None,NaN,None
14,5,1,"03 23, 2016",A265UZVOZWTTXQ,B000KPIHQ4,William_Jasper,Excellent insole with good support.,Five Stars,1458691200,None,None,None,None,NaN,None


In [25]:
df = df[["reviewerID", "overall", "asin"]]

### DF contains duplicates due to subtypes of products.
Only differences are colours etc. So the decision was made to drop these since they won't add any benefit when recommending products.

In [26]:
df.drop_duplicates(inplace=True)

In [27]:
df.reset_index(inplace=True)

In [28]:
df = df.pivot_table(index='reviewerID', columns='asin', values='overall')

# Changing the sparsity of the matrix

This could be done either by looping of each row until the desired amount of dropped cells is reached (implemented) or we could choose rows at random and drop cells untill the desired amount is reached. This way not only the first X rows will drop more than the others, but it would be divided

In [29]:
import numpy as np

In [30]:
sparsity = lambda dataf: dataf.isna().sum().sum() / (dataf.shape[0] * dataf.shape[1])

In [31]:
total = df.shape[0] * df.shape[1]

In [16]:
current_nan = df.isnull().sum().sum()

In [18]:
sparsity(df)

0.7583823295725409

In [19]:
desired_percentage = .8

In [20]:
cells_to_drop = (desired_percentage * total) - current_nan

In [21]:
cells_dropped = 0
while True:
    for index, row in df.iterrows():
        nans = np.where(row.notnull())[0]
        if not len(nans):
            continue
        drop = np.random.choice(nans)
        row[drop] = np.nan
        cells_dropped += 1
        if cells_dropped >= cells_to_drop:
            break    
    if cells_dropped >= cells_to_drop:
        break        


In [22]:
cells_dropped

524

In [23]:
sparsity(df)

0.8000158906721754